In [3]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt

import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.models as models
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.models as models
from torchvision.transforms import ToTensor
from torch import nn, optim
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader

import cv2
from PIL import Image
from transformers import ViTModel, ViTImageProcessor, ViTForImageClassification
from timm import create_model

from torch.utils.data.sampler import SubsetRandomSampler


import time
from tqdm import tqdm
import math
import random
import argparse
import logging
import json
import datetime

from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict

import tensorflow as tf
import keras
import ast
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, Input, Concatenate
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras import backend as K

logging.basicConfig(level=logging.INFO)

In [5]:
os.getcwd()
os.chdir('../..')
root_dir=os.getcwd()

In [6]:
print("Root Project Path : ", root_dir)
WEIGHT_DIR = os.path.join(root_dir, 'weights')
ROOT_DATA_DIR = os.path.join(root_dir, 'data', 'xview_building_damage')
MODEL_OUT = os.path.join(ROOT_DATA_DIR, 'model')
print("Root Data Path : ", ROOT_DATA_DIR)
TRAIN_DATA_DIR = os.path.join(ROOT_DATA_DIR, 'train')
print("Train Data Path : ", TRAIN_DATA_DIR)
CHALLENGE_DIR = os.path.join(ROOT_DATA_DIR, 'challenge')
TRAIN_DIR =  os.path.join(CHALLENGE_DIR, 'train')

TRAIN_DAT_DIR= os.path.join(TRAIN_DIR, 'disaster', 'hurricane-michael')
TEST_DIR =  os.path.join(CHALLENGE_DIR, 'test')

Test_disaster= os.path.join(TEST_DIR, 'disaster', 'hurricane-michael' )
TEST_DAT_DIR= os.path.join(TEST_DIR, 'disaster', 'hurricane-michael')

Root Project Path :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio
Root Data Path :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage
Train Data Path :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/train


In [7]:
LOG_DIR = os.path.join(root_dir, 'logs')
NUM_WORKERS = 4 
NUM_CLASSES = 4
BATCH_SIZE = 64
NUM_EPOCHS = 100 
LEARNING_RATE = 0.0001
RANDOM_SEED = 123
LOG_STEP = 150
LOG_DIR = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [8]:
damage_intensity_encoding = dict()
damage_intensity_encoding[5] = '5'
damage_intensity_encoding[4] = '4'
damage_intensity_encoding[3] = '3'
damage_intensity_encoding[2] = '2' 
damage_intensity_encoding[1] = '1' 
damage_intensity_encoding[0] = '0' 

In [9]:
train_csv = os.path.join(CHALLENGE_DIR, 'csv', 'hc_train_hurricane-michael.csv')
test_csv = os.path.join(CHALLENGE_DIR, 'csv','hc_test_hurricane-michael.csv')

df_test=pd.read_csv(test_csv)
df_test['labels'] = df_test['damage']
df_test['labels'].value_counts()
df_test.to_csv(os.path.join(CHALLENGE_DIR, 'csv','hc_test_labels_hurricane-michael.csv'))
test_csv_labels=os.path.join(CHALLENGE_DIR, 'csv','hc_test_labels_hurricane-michael.csv')

df_train=pd.read_csv(train_csv)
df_train['labels'] = df_train['damage']
df_train['labels'].value_counts()
df_train.to_csv(os.path.join(CHALLENGE_DIR, 'csv','hc_train_labels_hurricane-michael.csv'))
train_csv_labels=os.path.join(CHALLENGE_DIR, 'csv','hc_train_labels_hurricane-michael.csv')

print("Train DIR :", train_csv_labels) 
print("Train DIR :", test_csv_labels) 

class_weights=compute_class_weight(class_weight = 'balanced',classes= np.unique(df_train['labels'].to_list()), y = df_train['labels'].to_list())
d_class_weights = dict(enumerate(class_weights))
d_class_weights

df_train.columns

Train DIR : /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/csv/hc_train_labels_hurricane-michael.csv
Train DIR : /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/csv/hc_test_labels_hurricane-michael.csv


Index(['Unnamed: 0', 'sensor', 'provider_asset_type', 'gsd', 'capture_date',
       'off_nadir_angle', 'pan_resolution', 'sun_azimuth', 'sun_elevation',
       'target_azimuth', 'disaster', 'disaster_type', 'catalog_id',
       'original_width', 'original_height', 'width', 'height', 'id',
       'img_name', 'map_polygon', 'building_id', 'image_polygon',
       'feature_type', 'damage', 'dataset', 'image_id', 'is_pre_image',
       'is_post_image', 'mask_file_names', 'damage_class', 'labels'],
      dtype='object')

### Helper Methods

This code is adapted from XView2 Challenge Baseline https://github.com/DIUx-xView/xView2_baseline/blob/master/model/

In [10]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


###
# Creates data generator for validation set
###
def validation_generator(test_csv, test_dir):
    df = pd.read_csv(test_csv)
    df = df.replace({"labels" : damage_intensity_encoding })

    gen = keras.preprocessing.image.ImageDataGenerator(
                             rescale=1/255.)


    return gen.flow_from_dataframe(dataframe=df,
                                   directory=test_dir,
                                   x_col='building_id',
                                   y_col='labels',
                                   batch_size=BATCH_SIZE,
                                   shuffle=False,
                                   seed=RANDOM_SEED,
                                   class_mode="categorical",
                                   target_size=(128, 128))


###
# Applies random transformations to training data
###
def augment_data(df, in_dir):

    df = df.replace({"labels" : damage_intensity_encoding })
    data_gen_train = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             rescale=1/255.)
    train_generator = data_gen_train.flow_from_directory(in_dir, target_size=(128,128), batch_size=128, class_mode="categorical")
    return train_generator
    # return gen.flow_from_dataframe(dataframe=df,
    #                                directory=in_dir,
    #                                x_col='building_id',
    #                                y_col='labels',
    #                               # batch_size=BATCH_SIZE,
    #                                seed=RANDOM_SEED,
    #                                class_mode="categorical",
    #                                target_size=(128, 128))


# Run training and evaluation based on existing or new model
def train_model(train_data, train_csv, test_data, test_csv, model_out, model_in = None):

    model = generate_xBD_baseline_model()

    # Add model weights if provided by user
    if model_in is not None:
        model.load_weights(model_in)

    df = pd.read_csv(train_csv)
    class_weights=compute_class_weight(class_weight = 'balanced',classes= np.unique(df_train['labels'].to_list()), y = df_train['labels'].to_list())
    d_class_weights = dict(enumerate(class_weights))

    samples = df['building_id'].count()
    steps = np.ceil(samples/BATCH_SIZE)

    # Augments the training data
    train_gen_flow = augment_data(df, train_data)

    #Set up tensorboard logging
    tensorboard_callbacks = keras.callbacks.TensorBoard(log_dir=LOG_DIR,
                                                        batch_size=BATCH_SIZE)

    
    #Filepath to save model weights
    filepath = model_out + "-saved-model-{epoch:02d}-{accuracy:.2f}.hdf5"
    checkpoints = keras.callbacks.ModelCheckpoint(filepath,
                                                    monitor=['loss', 'accuracy'],
                                                    verbose=1,
                                                    save_best_only=False,
                                                    mode='max')

    #Adds adam optimizer
    adam = keras.optimizers.Adam(lr=LEARNING_RATE,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    weight_decay=0.0,
                                    amsgrad=False)


    model.compile(loss=ordinal_loss, optimizer=adam, metrics=['accuracy', f1])

    #Training begins
    model.fit(train_gen_flow,
                steps_per_epoch=steps,
                epochs=NUM_EPOCHS,
                workers=NUM_WORKERS,
                #use_multiprocessing=True,
                class_weight=d_class_weights,
                #callbacks=[tensorboard_callbacks, checkpoints],
                verbose=1)


    #Evalulate f1 weighted scores on validation set
    validation_gen = validation_generator(test_csv, test_data)
    predictions = model.predict(validation_gen)

    val_trues = validation_gen.classes
    val_pred = np.argmax(predictions, axis=-1)

    f1_weighted = f1_score(val_trues, val_pred, average='weighted')
    print(f1_weighted)

def ordinal_loss(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * keras.losses.categorical_crossentropy(y_true, y_pred )

def generate_xBD_baseline_model():
  weights = 'imagenet'
  inputs = Input(shape=(128, 128, 3))

  base_model = ResNet50(include_top=False, weights=weights, input_shape=(128, 128, 3))

  for layer in base_model.layers:
    layer.trainable = False

  x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Flatten()(x)

  base_resnet = base_model(inputs)
  base_resnet = Flatten()(base_resnet)

  concated_layers = Concatenate()([x, base_resnet])

  concated_layers = Dense(2024, activation='relu')(concated_layers)
  concated_layers = Dense(524, activation='relu')(concated_layers)
  concated_layers = Dense(124, activation='relu')(concated_layers)
  output = Dense(4, activation='relu')(concated_layers)

  model = Model(inputs=inputs, outputs=output)
  return model

In [11]:
parser = argparse.ArgumentParser(description='Run Building Damage Classification Training & Evaluation')
parser.add_argument('--train_data',
                    required=True,
                    metavar="/path/to/xBD_train",
                    help="Full path to the train data directory")
parser.add_argument('--train_csv',
                    required=True,
                    metavar="/path/to/xBD_split",
                    help="Full path to the train csv")
parser.add_argument('--test_data',
                    required=True,
                    metavar="/path/to/xBD_test",
                    help="Full path to the test data directory")
parser.add_argument('--test_csv',
                    required=True,
                    metavar="/path/to/xBD_split",
                    help="Full path to the test csv")
parser.add_argument('--model_in',
                    default=None,
                    metavar='/path/to/input_model',
                    help="Path to save model")
parser.add_argument('--model_out',
                    required=True,
                    metavar='/path/to/save_model',
                    help="Path to save model")

args = parser.parse_args(['--train_data', TRAIN_DAT_DIR , \
                          '--train_csv', train_csv_labels, \
                          '--test_data', TEST_DAT_DIR, \
                          '--test_csv', test_csv_labels, \
                          '--model_out', MODEL_OUT])


In [ ]:
train_model(args.train_data, args.train_csv, args.test_data, args.test_csv ,args.model_out)

Found 136802 images belonging to 4 classes.


Epoch 1/100
709/709 [==============================] - 1132s 2s/step - loss: 107.3141 - accuracy: 0.6624 - f1: 0.6618
Epoch 2/100
709/709 [==============================] - 1135s 2s/step - loss: 107.3675 - accuracy: 0.6632 - f1: 0.6632
Epoch 3/100
609/709 [========================>.....] - ETA: 3:02 - loss: 107.0224 - accuracy: 0.6643 - f1: 0.6643